<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">Chapitre 13 : Gestion des processus et des ressources</h1>

Considérons une activité simple sur ordinateur : Alice rédige un compte rendu pour un projet informatique.  
Elle a *ouvert* un logiciel de traitement de texte pour écrire le rapport. Son navigateur web est aussi ouvert avec divers onglets, l'un pointant vers Wikipedia, l'autre vers un moteur de recherche et un troisième vers un site de réseau social dont elle se sert pour partager son humeur avec ses camarades. Elle utilise un logiciel de dessin afin d'ajouter des illustrations à son compte-rendu. Son projet étant en Python, elle dispose aussi d'une fenêtre avec l'environnement Idle dans lequel elle exécute son programme afin d'en vérifier les résultats. Enfin, pour ne pas être perturbée, elle a mis des écouteurs sur ses oreilles et écoute de la
musique grâce au lecteur de musique de son ordinateur.

Tous ces programmes s'exécutent *en même temps*. Pourtant, si on se souvient de la façon dont sont construits les ordinateurs, ils ne disposent que d'un nombre limité de processeurs (la norme est de un à huit processeurs (on parle de *cœurs*, core en anglais) pour les ordinateurs personnels, téléphones et tablettes, de l'ordre de quelques dizaines de processeurs pour les machines de type serveur, et plusieurs milliers pour les super calculateurs). Or, comme on le sait, un programme n'est qu'une suite d'instructions en langage machine, ces dernières étant exécutées une à une par le processeur. Comment le processeur peut-il donc exécuter *en même temps* les instructions du programme de traitement de texte et celles du lecteur de musique?

Cette exécution **concurrente** de programmes est l'une des fonctionnalités de base offertes par les systèmes d'exploitation modernes. On parle alors de systèmes d'exploitation multitâches.  
Nous rappelons d'abord brièvement comment un programme est exécuté par le système d'exploitation, puis nous introduisons le principe de fonctionnement de l'**ordonnanceur de processus**,
la partie du système d'exploitation permettant l'exécution concurrente des programmes.

## L' ordonnanceur
### Rappel sur l'exécution d'un programme
Nous rappelons qu'un exécutable est un **fichier** (par exemple stocké sur le disque dur) contenant une suite d'instructions en langage machine (on appelle aussi souvent ces fichiers, des **binaires**, de l'anglais *binary*, abréviation de *binary code executable file*, fichier de code binaire exécutable). C'est
donc une suite d'octets que le processeur est capable de décoder et exécuter. Concrètement, lorsque l'on exécute un programme (par exemple, en cliquant sur l'icône du fichier exécutable ou en renseignant son chemin dans un terminal), le système d'exploitation effectue les actions suivantes :
1. le fichier contenant le programme (l'exécutable) est copié dans la mémoire RAM, à une certaine adresse `a`
2. le système d'exploitation écrit la valeur `a` dans le registre IP (Instruction Pointer)

Au prochain cycle d'horloge du processeur, ce dernier va alors lire l'instruction se trouvant à l'adresse `a` et l'exécuter.  
Une fois cela fait, il exécutera ensuite la seconde instruction et ainsi de suite.  
On rappelle que l'exécution d'une instruction se décompose, elle-même, en plusieurs sous-étapes effectuées au sein du processeur : 
* le chargement (récupérer l'instruction en mémoire)
* le décodage (déterminer dans la suite d'octets chargés quelle instruction ils encodent) 
* l'exécution proprement dite

### Interruptions
Même si elle est correcte, la description que nous avons faite de l'exécution d'un programme est incomplète.  
En effet, si rien de plus n'est fait, alors la seule chose que l'on peut attendre, c'est que le programme en question s'exécute jusqu'à sa dernière instruction, puis rende la main au système d'exploitation. Impossible alors de l'interrompre! Impossible aussi de pouvoir exécuter deux programmes en même temps.  
Pour pallier ce problème, les systèmes d'exploitation utilisent une fonctionnalité importante des processeurs modernes : la notion d'**interruption**.

Une interruption est un **signal** envoyé au processeur lorsqu'un événement se produit.  
Il existe plusieurs types d'interruptions.  
Certaines sont générées par le matériel (par exemple, un disque dur signale qu'il a fini d'écrire des octets, une carte réseau signale que des paquets de données arrivent, etc.).  
Lorsque le processeur reçoit une interruption, il **interrompt son exécution** à la fin de l'instruction courante et exécute un programme se trouvant à une adresse prédéfinie. Ce programme reçoit en argument une copie des valeurs courante des registres, ainsi qu'un code numérique lui permettant de savoir à quel type d'interruption il fait face. Ce programme spécial s'appelle le **gestionnaire d'interruption**. Il est installé à une certaine adresse mémoire par le système d'exploitation, très tôt après le démarrage de la machine.  
La réalité est un peu plus complexe : une unité spéciale du processeur, le contrôleur d'interruptions programmable ou PIC (pour l'anglais *Programmable Interrupt Controler*) dispose d'instructions assembleur dédiées permettant la configuration et la gestion des interruptions.

Parmi les interruptions matérielles, on retrouve les **interruptions d'horloge**. Le processeur génère de lui-même une interruption matérielle à intervalles de temps fixe. Historiquement, sur processeur Intel, cette interruption était levée toutes les 55 ms (environ 18 fois par seconde). Le gestionnaire d'interruption était donc appelé au moins toutes les 55 ms.  
De nos jours, les processeurs disposent d'horloges de haute précision capables d'émettre des interruptions avec une fréquence de 10 Mhz, donc toutes les 100 ns. Ces interruptions d'horloges, alliées au gestionnaire d'interruption, sont les pièces essentielles permettant d'exécuter des programmes de façon concurrente.

### Vocabulaire
Certains termes sont particulièrement importants pour la suite et doivent être définis précisément :
* **Exécutable** : un fichier binaire contenant des instructions machines directement exécutables par le processeur de la machine.
* **Processus** (*programme en cours d'exécution*) : un processus est le phénomène dynamique qui correspond à l'exécution d'un programme particulier.  
Le système d'exploitation identifie généralement les processus par un numéro unique.  
Un processus est décrit par :
    * l'ensemble de la mémoire allouée par le système pour l'exécution de ce programme (ce qui inclut le code exécutable copié en mémoire et toutes les données manipulées par le programme, sur la pile ou dans le tas)
    * l'ensemble des ressources utilisées par le programme (fichiers ouverts, connexions réseaux, etc.)
    * les valeurs stockées dans tous les registres du processeur.
* **Thread** (ou tâche) : exécution d'une suite d'instructions démarrée par un processus.  
Deux processus sont l'exécution de deux programmes (par exemple, un traitement de texte et un navigateur web).  
Deux threads sont l'exécution concurrente de deux suites d'instructions d'un même processus.  
Par exemple, pour un navigateur web, il peut y avoir un thread dont le rôle est de dessiner la page web dans une fenêtre et un autre thread dont le rôle est de télécharger un fichier sur lequel l'utilisateur a cliqué.  
La différence fondamentale entre processus et thread est que les processus ne partagent pas leur mémoire, alors que les threads, issus d'un même processus, peuvent accéder aux variables globales du programme et occupent le même espace en mémoire.
* **Exécution concurrente** : deux processus ou tâches s'exécutent de manière concurrente si les intervalles de temps entre le début et la fin de leur exécution ont une partie commune.
* **Exécution parallèle** : deux processus ou tâches s'exécutent en parallèle s'ils s'exécutent au même instant.  
Pour que deux processus s'éxcutent en parallèle, il faut donc plusieurs processeurs sur la machine.  
C'est une légère simplification : les processeurs modernes, même mono-cœur, disposent de plusieurs unités arithmétiques et logiques, ce qui leur permet d'exécuter certains opérations en parallèle. Par exemple, ils peuvent exécuter deux additions simultanément.

### Ordonnanceur du système d'exploitation
Comme nous l'avons vu, le système d'exploitation peut configurer une horloge et le gestionnaire d'interruption pour *reprendre la main*, c'est-à-dire exécuter du code qui lui est propre, à intervalles réguliers.  
Lorsqu'il s'exécute, il peut, entre autres choses, décider **à quel programme en cours d'exécution** il va rendre la main.  
Dans le scénario d'utilisation donné en introduction, l'ordonnanceur fonctionnera donc de la façon suivante:
1. Le programme *traitement de texte* est en cours d'exécution (l'utilisateur saisit du texte qui est affiché à l'écran, sauvegarde dans un fichier, etc.).
2. Une interruption d'horloge se déclenche.
3. Le code du gestionnaire d'interruption est appelé.  
Il reçoit en argument les valeurs qu'ont tous les registres avant le déclenchement de l'interruption (donc tout l'état *interne* du traitement de texte).
4. Le gestionnaire d'interruption sauvegarde ces registres à un endroit particulier de la mémoire.
5. Il choisit dans la liste des processus un autre processus, par exemple celui correspondant au navigateur web.
6. Il restaure les valeurs de tous les registres du processeur qu'il a sauvegardées la dernière fois qu'il a interrompu le navigateur web.  
Parmi ces registres sauvegardés, il y a notamment IP, l'adresse de la prochaine instruction à exécuter. Elle pointait alors vers une instruction du programme *navigateur web*.
7. Le gestionnaire d'interruption rend la main.  
La prochaine instruction à exécuter est celle du processus navigateur web, qui reprend son exécution, jusqu'à ce qu'il soit mis en pause par la prochaine interruption d'horloge.

Le fait que l'ordonnanceur interrompe un processus et sauve son état s'appelle une **commutation de contexte**.  
Afin de pouvoir choisir, parmi tous les processus, lequel exécuter lors de la prochaine interruption, le système d'exploitation conserve pour chaque processus une structure de données nommée PCB (pour l'anglais *Process Control Bloc* ou **bloc de contrôle du processus**). Le PCB est simplement une zone mémoire dans laquelle sont stockées diverses informations sur le processus.

| Nom        | Description                                                                    |
|:------------|:--------------------------------------------------------------------------------|
| PID        | Process ID, l'identifiant numérique du processus                               |
| État       | l'état dans lequel se trouve le processus                                      |
| Registres  | la valeur des registres lors de sa dernière interruption                       |
| Mémoire    | zone mémoire (plage d'adresses) allouée par le processus lors de son exécution |
| Ressources | liste des fichiers ouverts, connexions réseaux en cours d'utilisation, etc.    |

Pour choisir parmi les processus celui auquel il va donner la main, l'ordonnanceur conserve les PCB dans une structure de donnée (par exemple, une file). Le premier processus dans la file reprend son exécution. Lors de la prochaine interruption, il est mis en bout de file.  
Cette stratégie simple permet d'éviter qu'un processus monopolise tout le temps de calcul. En effet, avant qu'un processus mis en bout de file puisse s'exécuter de nouveau, il aura laissé une chance à tous les autres d'avoir la main.

### États des processus
Les interruptions d'horloges ne sont pas les seuls événements permettant d'interrompre les processus.  
Considérons le petit programme Python ci-dessous:

In [ ]:
texte = input("Saisir une phrase : ")
print("Votre phrase en majuscules: ", texte.upper())

Ce programme utilise la fonction `input` pour demander à l'utilisateur de saisir une phrase.  
Tant que l'utilisateur ne saisit rien, le programme ne peut pas avancer à l'instruction suivante. Le système d'exploitation (qui gère aussi le matériel et en particulier l'accès au clavier) peut suspendre le processus et le mettre en attente.  
De plus, il sait aussi qu'il est inutile de réveiller ce processus tant que l'utilisateur n'a pas interagi avec le clavier.  
Une telle interaction avec le clavier est signalée au processeur par une interruption, similaire aux interruptions d'horloges.  
De manière générale, lorsque des périphériques (carte réseau, disque dur, souris, clavier, etc.) veulent signaler au processeur qu'un événement est survenu, ils le feront au moyen d'une interruption. Le système d'exploitation aura alors l'occasion *de réveiller* l'un des processus parmi ceux qui attendaient un tel événement.

On voit donc que les processus peuvent être dans différents états.  
La plupart des systèmes d'exploitation utilisent principalement les états suivants:
* **Nouveau** : état d'un processus en cours de création.  
Le système d'exploitation vient de copier l'exécutable en mémoire et d'initialiser le PCB.
* **Prêt** : le processus peut être le prochain à s'exécuter.  
Il est dans la file des processus qui *attendent* leur tour et peuvent être choisis par l'ordonnanceur.
* **En exécution** : le processus est en train de s'exécuter.
* **En attente** : le processus est interrompu et en attente d'un événement externe (entrée/sortie, allocation mémoire, etc.).
* **Terminé** : le processus s'est terminé, le système d'exploitation est en train de désallouer les ressources que le processus utilisait.

Les états **Nouveau** et **Terminé** sont éphémères.  
En temps normal, l'état d'un processus variera entre prêt, en attente et en exécution.  

La figure suivante résume le cycle de vie d'un processus :

<div style="text-align: center">
   <img alt="Cycle de vie d'un processus" src="Images/processus-1.png" > 
</div>


On peut noter que quel que soit l'état dans lequel se trouve un processus, il peut se terminer de façon anormale.  
* S'il est en exécution, cela peut être dû à une erreur provoquée par le programme (lecture d'une adresse mémoire invalide, division par 0, etc.). 
* Si le processus est en attente d'une entrée-sortie, il est possible qu'il se produise une erreur matérielle (disque dur défectueux par exemple). 
* Enfin, si le processus est dans l'état prêt, il peut quand même se terminer de façon anormale si l'utilisateur qui a lancé le programme ou l'administrateur système décide de l'interrompre manuellement.

## Commandes Unix de gestion des processus
Dans les systèmes POSIX, la commande [`ps`](https://pubs.opengroup.org/onlinepubs/9699919799/utilities/ps.html) (pour l'anglais *process status* ou état des processus) permet d'obtenir des informations sur les processus en cours d'exécution.

```
utilisateur@machine:~$ ps -a -u -x
```
Les options `-a`, `-u` et `-x` permettent respectivement d'afficher tous les processus (et pas seulement ceux de l'utilisateur qui lance la commande), d'afficher le nom des utilisateurs (plutôt que leur identifiant numérique) et de compter aussi les processus n'ayant pas été lancés depuis un terminal (comme les *daemon* ou les processus lancés depuis une interface graphique).  
La commande affiche sur la sortie standard des informations sur les processus, comme par exemple :


```
USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.1 167880  7344 ?        Ss   09:26   2:40 /sbin/init sp
...
john        1438  0.0  0.0  11548  4952 tty2     Ss   15:45   0:00 bash
john        3537  5.4  3.6 998564 60764 ?        SI   15:12   9:11 /usr/bin/firefox
root        6524  0.0  0.0  29260  7780 ?        Ss   00:00   0:00 /usr/sbin/cupsd
john        6966  9.8  2.0 140692 24240 ?        SLI  15:41   2:56 /usr/bin/emacs
john        7490  0.0  0.0  11668  2704 tty2     R+   15:47   0:00 ps -a -u -x
...

```

Nous ne détaillons que les colonnes les plus importantes et, comme pour toute commande Unix, nous renvoyons le lecteur intéressé à la page de manuel de la commande accessible par la commande :

```
utilisateur@machine:~$ man ps
```

qui renvoie le manuel de la commance `ps` :

```sh
PS(1)                            User Commands                           PS(1)

NAME
       ps - report a snapshot of the current processes.

SYNOPSIS
       ps [options]

DESCRIPTION
       ps displays information about a selection of the active processes.  If
       you want a repetitive update of the selection and the displayed
       information, use top(1) instead.
       
...

```

* La colonne `USER` indique le nom de l'utilisateur qui a lancé le processus. 
* La colonne `PID` donne l'identifiant numérique du processus. 
* Les colonnes `%CPU` et `%MEM` indiquent respectivement le taux d'occupation du processeur et de la mémoire par le processus.  
Par exemple, dans l'affichage précédant, on peut voir que le processus `6966` occupe $9,8\%$ du temps de calcul du processeur et $2\%$ de la mémoire. En simplifiant un peu, on peut dire que sur les dernières $100$ secondes d'utilisation du système, $9,8$ secondes ont été passées à exécuter des instruction du processus `6966`. 
* La colonne `TTY` indique l'identifiant du terminal où le processus a été lancé.  
Un caractère `?` indique que le processus n'a pas été lancé depuis un terminal. 
* La colonne `STAT` indique l'état du processus (la première lettre en majuscule).  
Sur la plupart des systèmes Unix, les états sont:
    * `R` : *running* ou *runnable*, le processus est dans l'état prêt ou en exécution (la commande `ps` ne différencie pas ces deux états)
    * `S` : *sleeping*, le processus est **en attente**.
* Les colonnes `START` et `TIME` indiquent respectivement l'heure ou la date à laquelle le programme a été lancé et le temps cumulé d'exécution du processus correspondant (c'est-à-dire, le temps total pendant lequel le processus était dans l'état *en exécution*). 
* Enfin, la colonne `COMMAND` indique la ligne de commande utilisée pour lancer le programme (elle est tronquée dans notre exemple pour des raisons de place).

Une commande un peu plus conviviale est la commande `top`.  
Cette dernière affiche en temps réel des informations similaires à celles affichées par `ps`. Ces informations sont rafraîchies toutes les secondes.  
Cette commande est particulièrement précieuse pour essayer de déterminer quels processus occupent le plus le processeur ou ont alloué le plus de mémoire (sous le système Microsoft Windows, le gestionnaire de tâches joue un rôle similaire : il est accessible par la combinaison de touches `Ctrl-Alt-Supr`).  
L'affichage de la commande `top` est donné à la figure suivante.

```
utilisateur@machine:~$ top

top - 22:34:32 up 13:08,  1 user,  load average: 0,57, 0,55, 0,53
Tâches: 328 total,   2 en cours, 326 en veille,   0 arrêté,   0 zombie
%Cpu(s):  2,3 ut,  1,0 sy,  0,0 ni, 96,6 id,  0,0 wa,  0,0 hi,  0,0 si,  0,0 st
MiB Mem :   5879,0 total,   1499,0 libr,   2777,3 util,   1602,7 tamp/cache
MiB Éch:   2048,0 total,   1681,4 libr,    366,6 util.   2723,4 dispo Mem 

    PID UTIL.     PR  NI    VIRT    RES    SHR S  %CPU  %MEM    TEMPS+ COM.     
   3325 john      20   0 1523660  64696  29148 S   5,3   1,1  12:55.64 Xorg     
   3802 john      20   0 5729244 202928  45592 S   4,0   3,4  16:34.15 gnome-s+ 
  22863 john      20   0  829980  52420  38664 S   3,7   0,9   0:05.67 gnome-t+ 
  22224 john      20   0 3379920 430576 156760 S   3,3   7,2   2:59.20 chrome   
  23075 john      20   0   25,5g 261700 100492 S   3,3   4,3   5:51.24 chrome   
    208 root     -51   0       0      0      0 S   2,0   0,0   0:47.74 irq/31-+ 
  22534 john      20   0   39,5g 236192  93140 S   2,0   3,9   1:28.14 chrome   
  22877 john      20   0  363724  85120  20612 S   2,0   1,4   0:15.34 jupyter+ 
  22342 john      20   0 2345324 161496 104324 S   1,0   2,7   2:31.41 chrome   
  22344 john      20   0 1191244  98432  77084 S   1,0   1,6   0:22.37 chrome   
  22469 john      20   0   25,5g 140420  84416 S   0,7   2,3   0:09.15 chrome   
  22521 john      20   0   25,5g 106856  82676 S   0,7   1,8   0:06.59 chrome   
    552 root      -2   0       0      0      0 S   0,3   0,0   1:14.45 gfx      
  21962 root      20   0       0      0      0 R   0,3   0,0   0:02.83 kworker+ 
  22456 john      20   0   25,5g  97076  74044 S   0,3   1,6   0:03.18 chrome   
  23951 root      20   0       0      0      0 D   0,3   0,0   0:03.99 kworker+ 
  24493 root      20   0       0      0      0 I   0,3   0,0   0:01.29 kworker+
```

Si on souhaite interrompre un processus dont on connaît le PID, on peut utiliser la commande [`kill`](https://pubs.opengroup.org/onlinepubs/9699919799/utilities/kill.html), en lui spécifiant les numéros des processus que
l'on souhaite terminer.

```
utilisateur@machine:~$ kill 6966 3537
```

La commande ci-dessus va envoyer un **signal de terminaison** aux deux processus listés.  
* Pour les applications graphiques, ce signal est globalement équivalent à fermer la fenêtre principale de l'application.  
* Pour les applications en ligne de commande (qui s'exécutent dans un terminal en bloquant celui-ci), cela correspond à exécuter la combinaison de touche `Ctrl-C`.

Ce signal peut être intercepté par l'application et géré par cette dernière.  
Par exemple, un logiciel de traitement de texte peut, comme lorqu'on ferme la fenêtre, proposer à l'utilisateur de sauvegarder ses fichiers avant de quitter.  
Si on souhaite terminer immédiatement le processus sans que ce dernier puisse intercepter le signal, on peut passer l'option `-9` à la commande `kill`.

```
utilisateur@machine:~$ kill -9 6066 3537
```

Dans ce cas, les processus seront immédiatement terminés, sans qu'ils puissent exécuter la moindre instruction supplémentaire. En particulier, les fichiers non écrits seront perdus.  
Cette option est à utiliser en dernier recours, par exemple lorsque l'application ne *répond* plus et se comporte de manière anarchique (utilisation importante du processeur ou consommation de mémoire excessive).

## Programmation concurrente
### Processus concurrents
Comme nous l'avons vu, les processus d'un système s'exécutent de manière concurrente : leurs exécutions sont entrelacées.  
De plus, l'ordre dans lequel ils s'exécutent est hors de leur contrôle, car il est décidé par l'ordonnanceur de processus du système d'exploitation.  
Cette façon de fonctionner possède de nombreux avantages. 
* Elle permet l'exécution d'un très grand nombre de programmes sur une machine monoprocesseur. 
* Elle permet aussi d'optimiser les ressources de la machine.  
Par exemple, si un processus est en attente d'entrées-sorties, il est simplement mis en pause et le système peut utiliser le processeur pour effectuer un calcul utile. 
* Et même si tous les processus sont en attente d'un événement, le système peut alors décider dans ce cas de réduire la fréquence du processeur ou de le mettre partiellement en veille, ce qui permet d'économiser de l'énergie.  
Cet aspect est particulièrement important pour les systèmes mobiles ou embarqués.

Cependant, l'utilisation de systèmes multitâches n'est pas sans problème.  
En effet, lorsqu'un processus est interrompu, il ne s'en *rend pas compte*.

Dit autrement, lorsqu'un processus est interrompu, il reprendra son exécution exactement dans l'état où il s'était arrêté.  
Tant que ce processus manipule des objets visibles de lui seul (par exemple, des variables allouées sur la pile ou dans le tas), tout va bien. Mais si le processus accède à une **ressource partagée**, comme un fichier ou un périphérique matériel, alors de nombreux problèmes peuvent se produire.  
Considérons le programme  [ecritfichier.py](Fichiers/ecritfichier.py) suivant : 

In [ ]:
from os import getpid

pid = str(getpid())
with open ("test.txt", "w") as fichier:
    for i in range (1000):
        fichier.write(pid + " : " + str(i) + "\n")
        fichier.flush()

Ce programme importe la fonction utilitaire [`getpid`](https://docs.python.org/fr/3/library/os.html#os.getpid) du module [`os`](https://docs.python.org/fr/3/library/os.html).  
Celle-ci ne prend pas d'argument et renvoie simplement l'identifiant du processus dans lequel on se trouve.  
Le programme stocke dans une variable globale son identifiant de processus, converti en chaîne de caractères.  
Puis il ouvre le fichier `test.txt` en écriture (mode `'w'`, qui autorise l'écriture et vide le fichier s'il existe, plaçant le curseur interne en début de fichier).  
Le programme écrit ensuite 1000 lignes de la forme:

    12345 : 0
    12345 : 1
    ...
    12345 : 999

où `12345` est l'identifiant de processus déterminé au début du programme.  
La méthode `.flush()` de l'objet `fichier` assure que les caractères écrits avec la méthode `write` sont effectivement écrits sur le disque. 
En effet, l'écriture sur un disque dur étant coûteuse en temps, les fonctions d'écriture gardent en interne un tableau de caractères *tampon* dans lequel sont accumulés les caractères écrits au moyen de `.write()`. Seulement lorsque ce tableau est plein, les caractères sont effectivement écrits sur le disque et le tableau est vidé. 

Que se passe-t-il maintenant si on lance trois copies de ce programme simultanément?

```
utilisateur@machine:~$ python3 ecritfichier.py & python3 ecritfichier.py & python3 ecritfichier.py &

[1] 82208
[2] 82209
[3] 82210
[1]  Fini python3 code/ecritfichier.py
[2]- Fini python3 code/ecritfichier.py
[3]+ Fini python3 code/ecritfichier.py
```

L'utilisation du caractère `&` à la suite d'une commande bascule directement ce programme en arrière-plan et rend directement la main au terminal.  
On lance donc trois fois le programme. Le *shell* indique dans la console les identifiants des processus correspondants (ici respectivement `82208`, `82209` et `82210`). Les trois lignes contenant `Fini` sont écrites lorsque les programmes se terminent.  
Si on observe le contenu du fichier `test.txt`, on peut observer des lignes telles que :

    ...
    82209 : 840
    82209 : 841
    82208 : 842
    82210 : 843
    ...

On a donc l'impression que certaines lignes ont été écrites par certains processus uniquement.  
La réalité est plus complexe. Lors de l'ouverture du fichier, chaque processus initialise dans sa mémoire une variable de position (le curseur) contentant le décalage par rapport au début du fichier. Lorsque des caractères sont écrits au moyen de `.write()` le curseur est avancé d'autant de caractères.  
Un enchaînement d'exécutions de processus provoquant les écritures est représenté sur la figure suivante :

<div style="text-align: center">
   <img alt="Commutations de contexte entre processus" src="Images/concurrence-1.png" > 
</div>

Cette figure représente dans trois colonnes les états des trois processus, en particulier les valeurs internes du curseur dans le fichier `test.txt` et les chaînes qui sont écrites lorsque le processus a la main.  
La figure représente l'instant où le processus `82208` a la main. Ce dernier est à une certaine position dans le fichier (`9970`) et y écrit deux lignes.  
Il est alors interrompu par l'ordonnanceur qui provoque un changement de contexte et donne la main au processus `82210`.  
Ce dernier se trouvait à la même position dans le fichier, et donc au même indice de boucle (car les trois processus sont partis de l'indice `0` et écrivent exactement les mêmes nombres de caractères dans le fichier).  
Le processus `82210` écrit aussi deux lignes avant d'être interrompu.  
Le processus `82209` passe alors en exécution et peut écrire trois lignes. Comme c'est le dernier processus à écrire les lignes $840$ et $841$ ce sont ses écritures qui se retrouvent dans le fichier final.  
L'ordonnanceur rend la main au processus `82210` qui se retrouve dans l'état où sa variable de curseur vaut `9994` et sa variable `i` de boucle vaut `842`. Il va donc procéder à l'écriture de la chaîne `82210 : 842\n` dans le fichier, écrasant par là même la ligne qui avait été écrite à cette même position par le processus `82209`.  
Il est important de noter que cet *entrelacement* des instructions des trois processus est *aléatoire* ou plus exactement **non déterministe**. Si on réexécute les trois programmes, l'ordonnanceur peut décider de changer de contexte à d'autres moments, en fonction de divers paramètres (nombre de processus total en cours d'exécution, valeur des horloges, etc.) et on obtiendra un fichier différent.

### Interblocage
Un tel accès concurrent à une même ressource est souvent problématique.  
Certains périphériques matériels, en particulier, nécessitent un **accès exclusif**.  
Nous illustrons ce phénomène complexe avec les deux programmes suivant.
* **`enregistrer_micro`** : acquiert la carte son en accès exclusif (pour accéder au micro) et écrit les sons enregistrés sous un certain format (par exemple `mp3` ou `wav`) sur sa sortie standard et s'arrête lorsqu'il a écrit l'équivalent de 10 secondes de son.
* **`jouer_son`** : acquiert la carte son en accès exclusif (pour accéder au haut-parleur) et joue le contenu qu'il reçoit sur son entrée standard.

Même si nous ne donnons pas les détails du code (d'assez bas niveau) permettant de réaliser ces programmes, ils n'en restent pas moins assez réalistes.  
Les **cartes son** sont typiquement des périphériques ne pouvant être utilisés que par au plus un processus (dans les systèmes d'exploitation actuels, un processus particulier de type daemon agit comme un *serveur de sons* : il est le seul à accéder directement à la carte son, les autres processus communiquent avec lui pour lui demander d'enregistrer ou jouer des sons).  
Ces deux programmes peuvent être exécutés séquentiellement de la façon suivante :
```
utilisateur@machine:~$ enregistrer_micro > message.mp3
utilisateur@machine:~$ cat message.mp3 | jouer_son
```
La première ligne de commande appelle le programme `enregistrer_micro` et enregistre (au moyen d'une redirection) les octets émis sur sa sortie standard dans le fichier `message.mp3`.  
La deuxième commande utilise l'utilitaire `cat` pour afficher le contenu du fichier `message.mp3` sur la sortie standard puis redirige cette dernière au moyen de l'opérateur `|` sur l'entrée standard de la commande `jouer_son`.  

Que se passe-t-il maintenant si on enchaîne directement les deux commandes?

```
utilisateur@machine:~$ enregistrer_micro | jouer_son
```
Le premier programme ouvre la carte son. Il commence ensuite à envoyer des données sur sa sortie standard.  
Le second programme tente alors d'ouvrir la carte son et se retrouve bloqué. Il ne peut continuer son exécution tant que la carte son n'est pas disponible.  
Comme le programme `jouer_son` est bloqué, il ne lit aucun octet sur son entrée standard. Comme il ne lit aucun octet, le premier programme se retrouve lui aussi bloqué lorsqu'il tente d'écrire sur sa sortie standard (puisque personne ne lit, les octets s'accumulent dans une zone mémoire tampon et le programme est bloqué quand cette dernière est pleine).  
Les deux processus correspondant à ces deux programmes sont en **interblocage** :
* le processus de `jouer_son` attend que la carte son soit libre pour progresser et lire son entrée standard
* le processus d'`enregistrer_micro` attend que quelqu'un consomme son entrée standard pour progresser et libérer la carte son

L'interblocage est le grand danger de la programmation concurrente.  
Dans notre exemple, l'utilisateur constaterait juste que les deux programmes ne font rien et ne rendent pas la main.  

Il existe quatre conditions nécessaires à la présence d'un interblocage, appelées **conditions de Coffman**, du nom d'Edward Grady Coffman Jr. (1934-), informaticien américain qui les a décrites en premier en 1971.
1. **Exclusion mutuelle** : au moins une ressource du système doit être en accès exclusif.
2. **Rétention et attente** : un processus détient une ressource et demande une autre ressource détenue par un autre processus.
3. **Non préemption** : une ressource ne peut être rendue que par un processus qui la détient (et ne peut pas être *préemptée* ou acquise de force par un autre processus).
4. **Attente circulaire** : l'ensemble des processus bloqués $P_1$, ... , $P_n$ sont tels que $P_1$ attend une ressource tenue par $P_2$, ... , $P_n$ attend une ressource détenue par $P_1$.

Dans notre exemple précédent, les quatre conditions sont remplies.  
* Il existe une ressource à accès exclusif, la carte son (1).  
* Le processus du programme `enregistrer_micro` détient la carte son et veut, en plus, pouvoir écrire sur sa sortie (2).  
* Nous avons fait l'hypothèse que `jouer_son` est bloqué et ne peut pas acquérir la carte son (3).  
* Le programme `enregistrer_micro` attend le déblocage de son entrée standard bloquée par `jouer_son` et ce dernier attend le déblocage de la carte son détenue par `enregistrer_micro` (4).

Il existe plusieurs stratégies permettant d'éviter les interblocages ou de les détecter et de les résoudre. 
Dans le cadre générique des systèmes d'exploitation, et des programmes utilisateurs, la solution souvent retenue est la plus simple, à savoir interrompre les programmes (par exemple au moyen de la commande `kill`).

### Programmation concurrente en Python
Afin d'illustrer les problématiques d'interblocage dans un cadre plus contrôlé que dans un système d'exploitation, nous donnons ici une introduction à la **programmation multithread** en Python.  
Comme nous l'avons déjà expliqué, un thread est un *sous-processus* démarré par un processus et s'exécutant de manière concurrente avec le reste du programme. Le module [`threading`](https://docs.python.org/fr/3/library/threading.html) de la bibliothèque standard Python permet de démarrer des **threads**.  
Nous illustrons son utilisation au moyen du programme [`pr_threads.py`](Fichiers/pr_threads.py) suivant :

In [ ]:
import threading

def hello(n):
    for i in range(5):
        print ("Je suis le thread", n, "et ma valeur est", i)
    print ("------ Fin du Thread ", n)

for n in range(4):
    t = threading.Thread(target=hello, args=[n])
    t.start()

Ce programme définit une fonction `hello` prenant en argument un entier représentant l'identifiant du thread dans lequel on se trouve.  
Cette fonction effectue ensuite une boucle pour `i` allant de `0` à `4` et écrit à chaque tour de boucle la valeur de `n` et de `i`.  
La fonction imprime ensuite un message indiquant qu'elle a terminé la boucle puis se termine.  
Le programme principal effectue une boucle et appelle quatre fois (pour `n` entre `0` et `3`) l'expression `threading.Thread(target = hello, args = [n])`.  
Cette dernière crée un objet de type [`Thread`](https://docs.python.org/fr/3/library/threading.html#threading.Thread).  
L'argument nommé `target` doit être une fonction et l'argument `args` un tableau des arguments qui seront passés à la fonction.  
La variable `t` contient l'objet `Thread` créé.  
La méthode [`.start()`](https://docs.python.org/fr/3/library/threading.html#threading.Thread.start) lance l'exécution de la fonction en tâche de fond. Cette méthode rend directement la main et le programme principal continue de s'exécuter de façon concurrente au thread démarré.  
Le programme ci-dessus, une fois exécuté, comporte alors cinq threads : ceux démarrés par `.start()` et le thread principal.  
Voici un affichage possible pour ce programme :

    Je suis le thread 0 et ma valeur est 0
    Je suis le thread 0 et ma valeur est 1
    Je suis le thread 0 et ma valeur est 2
    Je suis le thread 1 et ma valeur est 0
    Je suis le thread 1 et ma valeur est 1
    Je suis le thread 1 et ma valeur est 2
    Je suis le thread 1 et ma valeur est 3
    Je suis le thread 2 et ma valeur est 0
    Je suis le thread 2 et ma valeur est 1
    Je suis le thread 2 et ma valeur est 2
    Je suis le thread 2 et ma valeur est 3
    Je suis le thread 2 et ma valeur est 4
    ------ Fin du Thread 2
    Je suis le thread 1 et ma valeur est 4
    ------ Fin du Thread 1
    Je suis le thread 3 et ma valeur est 0
    Je suis le thread 3 et ma valeur est 1
    Je suis le thread 3 et ma valeur est 2
    Je suis le thread 3 et ma valeur est 3
    Je suis le thread 3 et ma valeur est 4
    ------ Fin du Thread 3
    Je suis le thread 0 et ma valeur est 3
    Je suis le thread 0 et ma valeur est 4
    ------ Fin du Thread 0
    
Comme pour les processus, les threads alternent leur exécution au gré des commutations de contexte.  
Deux exécutions successives donnent des affichages différents. L'ordre dans lequel sont démarrés les threads ne donne aucune indication sur l'ordre dans lequel ils peuvent se terminer (dans l'exemple ci-dessus, le thread 0, démarré en premier se termine le dernier).  

On peut trouver étonnant que les threads ne s'interrompent pas les un les autres au milieu d'une ligne (ce phénomène s'explique par la notion de **verrous cachés**).

Les threads peuvent servir à illustrer les problèmes de concurrence et d'interblocage.  
Considérons le programme [`pb_threads.py`](Fichiers/pb_threads.py) suivant :

In [ ]:
import threading
COMPTEUR = 0

def incrc():
    global COMPTEUR
    for c in range(100000):
        v = COMPTEUR
        COMPTEUR = v + 1

th = []
for n in range(4):
    t = threading.Thread(target = incrc, args = [])
    t.start()
    th.append(t)

for t in th:
    t.join()
print ("valeur finale", COMPTEUR)

Ce dernier définit une variable globale `COMPTEUR`.  
La fonction `incrc` est similaire à la fonction `hello` du programme précédent. Elle ne prend pas d'argument mais exécute `100000` itérations d'une boucle qui incrémente la variable globale `COMPTEUR`.  
Le programme principal déclare un tableau vide `th`.  
Il démarre ensuite quatre threads et stocke les objets correspondants dans le tableau `th`, après les avoir démarrés.  
Enfin, pour chacun des objets `Thread` stockés, la méthode [`.join()`](https://docs.python.org/fr/3/library/threading.html#threading.Thread.join) est appelée. Cette dernière permet d'attendre que le thread auquel on l'applique soit terminé. Si le thread est déjà terminé, la méthode se termine immédiatement.  
Enfin, le programme imprime la valeur finale contenue dans le compteur.  
Comme on a démarré quatre threads, et que chacun incrémente la valeur `100000` fois, on s'attend à ce que l'affichage final soit `400000`.  
Cependant, si on exécute le programme plusieurs fois, on peut constater qu'il n'affiche pas toujours le nombre attendu :

```
utilisateur@machine:~$ python3 pb_threads.py
valeur finale 344447
```

Que se passe-t-il?  
Considérons les quatre threads $t_0$ à $t_3$.  
Supposons que $t_0$ soit en exécution et que la valeur de `COMPTEUR` soit `42`. Si $t_0$ est interrompu juste après avoir exécuté `v = COMPTEUR`, alors sa variable locale `v` contient la valeur `42`.  
La commutation de contexte donne la main à $t_1$, qui exécute `v = COMPTEUR` suivi de `COMPTEUR = v + 1` avant d'être lui-même interrompu.  
La valeur de compteur continue d'augmenter lors des commutations de contexte suivantes avec $t_2$ et $t_3$ jusqu'à avoir `COMPTEUR` valant `50`.  
Lorsque $t_0$ reprend enfin la main, il continue là où il s'était arrêté et exécute donc `COMPTEUR = v + 1`, où la valeur de `v` est `42`!  
Le thread $t_0$ va donc écraser la valeur `50` avec la valeur `43`. 

Pour corriger ce problème, il nous faut donc garantir l'**accès exclusif** à la variable `COMPTEUR` entre sa lecture et son écriture.  
On peut, pour cela, utiliser un **verrou**. Un verrou est un objet que l'on peut essayer d'acquérir. 
* Si on est le premier à faire cette demande, on acquiert le verrou. On peut le rendre à tout moment. 
* Si en revanche quelqu'un d'autre tient le verrou, alors on est bloqué jusqu'à ce qu'il soit libéré. 

Des verrous munis de ces deux opérations sont disponibles dans le module `threading` avec le constructeur [`Lock`](https://docs.python.org/fr/3/library/threading.html#threading.Lock).  
Une fois l'objet verrou construit, on peut tenter de l'acquérir avec la méthode [`.acquire()`](https://docs.python.org/fr/3/library/threading.html#threading.Lock.acquire) et on peut le rendre avec la méthode [`.release()`](https://docs.python.org/fr/3/library/threading.html#threading.Lock.release).  
Une manière de corriger le programme [`pb_threads.py`](Fichiers/pb_threads.py) est la suivante :

In [ ]:
import threading
verrou = threading.Lock()
COMPTEUR = 0

def incrc():
    global COMPTEUR
    for c in range(100000):
        verrou.acquire()
        v = COMPTEUR
        COMPTEUR = v + 1
        verrou.release()

th = []
for n in range(4):
    t = threading.Thread(target = incrc, args = [])
    t.start()
    th.append(t)

for t in th:
    t.join()
print ("valeur finale", COMPTEUR)

Avant toute tentative de lecture, on essaye d'acquérir le verrou.  
Une fois ce dernier acquis, le thread courant a la garantie qu'il est le seul à exécuter son code, jusqu'à l'instruction `verrou.release()`. Une telle portion de code protégée par un verrou s'appelle une **section critique**.  
Attention, cela ne signifie pas que le thread ne peut pas être interrompu entre les lignes `v = COMPTEUR` et `COMPTEUR = v + 1`. Cela signifie seulement que les autres threads, s'il reprennent la main, ne sont pas eux-mêmes en section critique (ils sont forcément ailleurs dans leur code, probablement bloqués sur l'instruction `verrou.acquire()`).  
On remarque qu'il est important que tous les threads manipulent le même verrou. C'est pour cela qu'il a été défini dans une variable globale accessible depuis tous les threads.  
En dernière remarque, on peut ajouter que le programme aurait été tout aussi faux si on avait écrit `COMPTEUR = COMPTEUR + 1` ou même `COMPTEUR += 1`. En effet, une instruction d'un langage de haut niveau comme Python est en fait décomposée en de nombreuses instructions machines : lecture de la valeur du compteur en mémoire, addition et écriture en mémoire de la nouvelle valeur.  
Le programme aurait donc pu être interrompu entre deux de ces instructions machines.

L'utilisation de plusieurs verrous rend les interblocages possibles.  
Il conviendra donc d'être très prudent lorsque l'on manipule deux verrous à la fois.  
On illustre ce problème avec le programme suivant :

In [ ]:
import threading

verrou1 = threading.Lock()
verrou2 = threading.Lock()

def f1():
    verrou1.acquire()
    print ("Section critique 1.1")
    verrou2.acquire()
    print ("Section critique 1.2")
    verrou2.release()
    verrou1.release()

def f2():
    verrou2.acquire()
    print ("Section critique 2.1")
    verrou1.acquire()
    print ("Section critique 2.2")
    verrou1.release()
    verrou2.release()

t1 = threading.Thread(target = f1, args = [])
t2 = threading.Thread(target = f2, args = [])
t1.start()
t2.start()

Ce dernier déclare deux verrous, utilisés de façon symétrique par deux fonctions `f1` et `f2`.  
La fonction `f1` essaye d'acquérir d'abord `verrou1` puis `verrou2`, alors que `f2` essaye de les acquérir dans l'ordre inverse.  
Si on exécute ce programme, il a de grandes chances de se retrouver bloqué.  
Considérons l'exécution suivante:
* Le thread `t1` a la main. Il s'exécute jusqu'à son premier affichage (avant la tentative d'acquisition de `verrou2`).
* Le thread `t2` prend la main. Il s'exécute, acquiert `verrou2` qui est toujours libre, puis bloque sur l'acquisition de `verrou1`.
* Le thread `t1` reprend la main, il bloque alors sur l'acquisition de `verrou2` (tenu par `t2`).

Chaque thread détient un verrou et attend l'autre. Ils sont en interblocage.  
Cependant, le problème ne se manifeste que si les exécutions se font dans cet ordre.  
Si la commutation de contexte intervient après que `f1` a acquis `verrou2`, alors `t1` peut se terminer sans bloquer.  

Dans des programmes complexes, les situations d'interblocage sont particulièrement difficile à tester et à corriger. En effet, à cause du non déterminisme de l'ordonnancement des threads et des processus, il se peut que le programme se comporte bien lors de la phase de test et ne se bloque que lorsqu'il est exécuté en conditions réelles.

#### Verrous cachés. 
Considérons de nouveau le programme : 

In [ ]:
import threading

def hello(n):
    for i in range(5):
        print ("Je suis le thread", n, "et ma valeur est", i)
    print ("------ Fin du Thread ", n)

for n in range(4):
    t = threading.Thread(target=hello, args=[n])
    t.start()

Puisque les commutations de contextes peuvent intervenir à tout moment, il peut sembler étrange que les affichages des threads s'entrelacent *proprement*.  
En effet, l'affichage du programme donne l'impression que les threads ne sont jamais interrompus au milieu d'une ligne. Ce comportement est du au fait que, par défaut, la sortie standard de Python possède un **buffer** (ou zone tampon). Ce dernier n'est rien d'autre qu'un tableau d'octets. Chaque appel à `print` n'écrit pas directement dans la console, ce qui serait inefficace, mais ajoute les caractères à la suite dans le buffer.  
Lorsque ce dernier est plein, il est affiché d'un seul coup en utilisant une bas niveau du système d'exploitation.  
Pour les fichiers textes (comme la sortie standard) le buffer est vidé à chaque retour chariot.  
Hors, ce buffer n'est rien d'autre qu'un tableau d'octets, accompagnés d'entiers représentant sa taille, et la position du dernier caractère écrit.

Pour éviter le phénomène illustré dans le programme suivant :

In [ ]:
import threading
COMPTEUR = 0

def incrc():
    global COMPTEUR
    for c in range(100000):
        v = COMPTEUR
        COMPTEUR = v + 1

th = []
for n in range(4):
    t = threading.Thread(target=incrc, args=[])
    t.start()
    th.append(t)

for t in th:
    t.join()
print ("valeur finale", COMPTEUR)

les mises à jour du buffer ainsi que les modifications de ces entiers sont **protégées par un verrou**.  
En effet, il est naturel de vouloir utiliser `print` depuis plusieurs threads différents. Si on écrit une chaîne se terminant avec retour chariot (comportement par défaut de `print`) il se produit donc le phénomène suivant :
* acquisition d'un verrou associé au buffer
* écriture de tous les caractères dans le buffer
* arrivé au retour chariot, envoyer lecontenu du buffer dans la console, puis le vider
* relâcher le verrou.

Deux threads ne peuvent donc pas mélanger leur ligne, puisque l'écriture d'une ligne devient une section critique.  

On peut obtenir un comportement plus *réaliste* en désactivant les buffers d'écriture globalement, avec l'option `-u` de l'interprète Python:

```
utilisateur@machine:~$ python3 -u pr_thread.py
Je suis le thread 0 Je suis le thread 1Je
suis le thread 2 et ma valeur estet
ma valeur est et ma valeur estO
Je suis le thread 00
et ma valeur estJe suis le thread 1 20
et ma valeur estJe suis le thread 11
Je suis le threadet ma valeur est 21
```

## Exercices 
### Exercice 1
On suppose qu'Alice exécute dans son terminal la commande `ps -a -u -x`.  
Le processus correspondant à cette commande fera partie des processus affichés dans la sortie.  

Dire quel sera l'état de ce processus (R ou S) et justifier.

### Exercice 2
On considère le programme suivant.

In [ ]:
import threading
COMPTEUR = 0

def incrc():
    global COMPTEUR
    for c in range(100000):
        v = COMPTEUR
        COMPTEUR = v + 1

th = []
for n in range(4):
    t = threading.Thread(target = incrc, args = [])
    t.start()
    th.append(t)

for t in th:
    t.join()
print ("valeur finale", COMPTEUR)

Pour chacune des affirmations suivantes, dire si elle est vraie ou fausse et justifier.
1. Le programme affiche toujours 400000.
2. Le programme peut afficher un nombre plus petit que 400000.
3. Le programme peut afficher un nombre plus grand que 400000.
4. Si on ajoute `t.join()` après `t.start()`, le programme affiche toujours 400 000.
5. Si on transforme le corps de la boucle de la fonction `incrc()` par
```python
verrou.acquire()
v = COMPTEUR
verrou.release()
verrou.acquire()
COMPTEUR = v + 1
verrou.release()
```
en ayant définie un variable globale `verrou = threading.Lock()` , alors le programme affiche toujours 400000.

### Exercice 3
On considère la situation de la figure suivante :

<div style="text-align: center">
   <img alt="Exercice" src="Images/blocage-1.png" > 
</div>

dans laquelle quatre voitures sont bloquées à une intersection.  
Montrer qu'il s'agit d'un interblocage, c'est-à-dire que les quatre conditions de Coffman sont réunies.  
On indiquera précisément quelles sont les ressources et les processus dans cette situation.  
On fera l'hypothèse que les conducteurs sont raisonnables, i.e qu'ils ne veulent pas provoquer d'accident.

### Exercice 4
Considérons un petit système embarqué : un petit ordinateur relié à trois LED A, B et C.  
Une LED peut être éteinte ou allumée et on peut configurer sa couleur.  
On dispose de trois programmes qui affichent des signaux lumineux en faisant clignoter les LED.

Chaque programme possède une LED primaire et une LED secondaire. 

* Le programme $P_1$ affiche ses signaux sur A (primaire) et B (secondaire) en vert.  
* Le programme $P_2$ affiche ses signaux sur B (primaire) et C (secondaire) en orange.  
* Le programme $P_3$ affiche ses signaux sur C (primaire) et A (secondaire) en rouge.

Comme les LED ne supportent pas d'être configurées dans deux couleurs en même temps, le système propose deux primitives `acquerirLED(nom)` et `rendreLED(nom)` qui permettent respectivement d'acquérir et de relâcher une LED.  
Si une LED est déjà acquise, alors `acquerirLED()` bloque.

On suppose que chacun des trois programmes $P_1$, $P_2$ et $P_3$ effectue les actions suivantes en boucle:
1. acquérir sa LED primaire
2. acquérir sa LED secondaire
3. configurer les couleurs
4. émettre des signaux
5. rendre la LED secondaire
6. rendre la LED primaire
7. recommencer en 1
Montrer qu'il existe un entrelacement des exécutions qui place $P_1$, $P_2$ et $P_3$ en interblocage.

### Exercice 5
Écrire un programme Python simulant le code de l'exercice précédent.  
On pourra s'inspirer du programme [`interblocage.py`](Fichiers/interblocage.py).  
Constater qu'en exécutant suffisamment de fois votre programme il se bloque.

Indication : afin de laisser plus de chance au système de changer de contexte, on pourra mettre des affichages (comme dans le programme [`interblocage.py`](Fichiers/interblocage.py)) juste après l'acquisition d'un verrou. En effet, l'écriture dans la console passe le thread courant en attente, le temps que les écritures soient effectuées, ce qui laisse une opportunité à l'ordonnanceur de choisir un autre thread ou un autre processus.

### Exercice 6
Si deux transactions travaillent sur un même objet (par exemple sur une même table), alors la seconde est bloquée jusqu'à ce que la première soit terminée.

Nous pouvons maintenant expliciter ce mécanisme : lors qu'une transaction accède à une table, elle tente de prendre un verrou sur cette dernière.

Les verrous sont relâchés au moment du `COMMIT` ou `ROLLBACK`.

La réalité est plus complexe. Pour des raisons de performances, les SGBD tentent d'acquérir les verrous le plus tard possible et de les relâcher le plus tôt possible, mais nous n'entrons pas dans ces détails. 

On considère deux tables :

```sql
CREATE TABLE T (num INTEGER);
CREATE TABLE S (num INTEGER);
INSERT INTO T VALUES (1000);
INSERT INTO S VALUES (1000);
```

qui peuvent représenter de façon simplifiée des comptes en banque.  

Considérons les deux transactions suivantes :

```sql
START TRANSACTION;
    UPDATE T SET num = num + 100;
    UPDATE S SET num = num - 100;
COMMIT;
```

et

```sql
START TRANSACTION;
    UPDATE S SET num = num + 100;
    UPDATE T SET num = num - 100;
COMMIT;
```

La première simule un virement de `S` vers `T` et la seconde un virement de `T` vers `S`.  

Montrer qu'il s'agit d'un interblocage, c'est-à-dire que les quatre conditions de Coffman sont réunies.

## Liens :
* Interstices : [Le ballet des processus dans un système d’exploitation](https://interstices.info/le-ballet-des-processus-dans-un-systeme-dexploitation/)
* Interstices : [À quoi sert un système d’exploitation ?](https://interstices.info/a-quoi-sert-un-systeme-dexploitation/)
* Société Informatique de France : [Brève histoire des systèmes d'exploitation](https://www.societe-informatique-de-france.fr/wp-content/uploads/2016/04/1024-no8-histoire-SE.pdf)
* [The Deadlock Empire](https://deadlockempire.github.io/)